## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 100

application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")


# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = classification_counts[classification_counts > 1]
print(classification_counts_gt1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 100

classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_df = pd.get_dummies(application_df)


In [10]:
numeric_df 

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = numeric_df.drop('IS_SUCCESSFUL', axis=1).values  # Features (excluding the target)
y = numeric_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=80, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(Dense(units=30, activation="relu"))

# Output layer
nn.add(Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4080      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6541 (25.55 KB)
Trainable params: 6541 (25.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(optimizer="adam", 
           loss="binary_crossentropy", 
           metrics=["accuracy"])

In [15]:
# Train the model
history = nn.fit(
    X_train_scaled,
    y_train,
    epochs=100,  # number of passes through the entire dataset
    batch_size=32,  
    validation_split=0.2,  
    verbose=1  
)

Epoch 1/100
686/686 [==============================] - 3s 2ms/step - loss: 0.5754 - accuracy: 0.7142 - val_loss: 0.5525 - val_accuracy: 0.7356
Epoch 2/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5553 - accuracy: 0.7274 - val_loss: 0.5492 - val_accuracy: 0.7336
Epoch 3/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5520 - accuracy: 0.7277 - val_loss: 0.5470 - val_accuracy: 0.7374
Epoch 4/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5502 - accuracy: 0.7306 - val_loss: 0.5468 - val_accuracy: 0.7385
Epoch 5/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7296 - val_loss: 0.5440 - val_accuracy: 0.7380
Epoch 6/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7313 - val_loss: 0.5459 - val_accuracy: 0.7405
Epoch 7/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7306 - val_loss: 0.5448 - val_accuracy: 0.7400

Epoch 58/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7380 - val_loss: 0.5447 - val_accuracy: 0.7391
Epoch 59/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5350 - accuracy: 0.7379 - val_loss: 0.5451 - val_accuracy: 0.7387
Epoch 60/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7378 - val_loss: 0.5435 - val_accuracy: 0.7385
Epoch 61/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5349 - accuracy: 0.7374 - val_loss: 0.5461 - val_accuracy: 0.7391
Epoch 62/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7371 - val_loss: 0.5452 - val_accuracy: 0.7341
Epoch 63/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7370 - val_loss: 0.5436 - val_accuracy: 0.7400
Epoch 64/100
686/686 [==============================] - 1s 2ms/step - loss: 0.5345 - accuracy: 0.7381 - val_loss: 0.5462 - val_accuracy:

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5628 - accuracy: 0.7276 - 238ms/epoch - 1ms/step
Loss: 0.5628069639205933, Accuracy: 0.7275510430335999


In [17]:
# Export our model to HDF5 file
nn.save("my_model.h5")


C:\Users\bobda\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10)

history = nn.fit(
    X_train_scaled,
    y_train,
    epochs=200,  
    batch_size=32,  
    validation_split=0.2,  
    verbose=1,
    callbacks=[early_stop]
)


Epoch 1/200
686/686 [==============================] - 1s 2ms/step - loss: 0.5323 - accuracy: 0.7385 - val_loss: 0.5489 - val_accuracy: 0.7391
Epoch 2/200
686/686 [==============================] - 1s 2ms/step - loss: 0.5320 - accuracy: 0.7387 - val_loss: 0.5529 - val_accuracy: 0.7367
Epoch 3/200
686/686 [==============================] - 1s 2ms/step - loss: 0.5325 - accuracy: 0.7382 - val_loss: 0.5476 - val_accuracy: 0.7385
Epoch 4/200
686/686 [==============================] - 1s 2ms/step - loss: 0.5323 - accuracy: 0.7384 - val_loss: 0.5486 - val_accuracy: 0.7392
Epoch 5/200
686/686 [==============================] - 1s 2ms/step - loss: 0.5318 - accuracy: 0.7385 - val_loss: 0.5465 - val_accuracy: 0.7389
Epoch 6/200
686/686 [==============================] - 1s 2ms/step - loss: 0.5324 - accuracy: 0.7387 - val_loss: 0.5468 - val_accuracy: 0.7389
Epoch 7/200
686/686 [==============================] - 1s 2ms/step - loss: 0.5317 - accuracy: 0.7389 - val_loss: 0.5478 - val_accuracy: 0.7382

In [19]:
# Define the model
number_input_features = len(X_train_scaled[0])
nn = Sequential()

# First hidden layer with dropout and batch normalization
nn.add(Dense(units=100, input_dim=number_input_features, activation="relu"))
nn.add(Dropout(0.2))
nn.add(BatchNormalization())

# Second hidden layer with dropout and batch normalization
nn.add(Dense(units=50, activation="relu"))
nn.add(Dropout(0.2))
nn.add(BatchNormalization())

# Optionally, a third hidden layer
nn.add(Dense(units=25, activation="relu"))

# Output layer
nn.add(Dense(units=1, activation="sigmoid"))

In [20]:
# Compile the model with a custom learning rate
opt = Adam(learning_rate=0.001)
nn.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])


In [21]:
# Use early stopping based on validation loss
early_stop = EarlyStopping(monitor='val_loss', patience=10)

In [22]:
# Train the model
history = nn.fit(
    X_train_scaled,
    y_train,
    epochs=200,  
    batch_size=32,  
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stop]
)

Epoch 1/200
686/686 [==============================] - 4s 3ms/step - loss: 0.6033 - accuracy: 0.6919 - val_loss: 0.5521 - val_accuracy: 0.7314
Epoch 2/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5729 - accuracy: 0.7173 - val_loss: 0.5484 - val_accuracy: 0.7354
Epoch 3/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5661 - accuracy: 0.7198 - val_loss: 0.5449 - val_accuracy: 0.7400
Epoch 4/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5638 - accuracy: 0.7232 - val_loss: 0.5435 - val_accuracy: 0.7372
Epoch 5/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5607 - accuracy: 0.7227 - val_loss: 0.5440 - val_accuracy: 0.7378
Epoch 6/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5598 - accuracy: 0.7236 - val_loss: 0.5448 - val_accuracy: 0.7400
Epoch 7/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5572 - accuracy: 0.7251 - val_loss: 0.5441 - val_accuracy: 0.7380

In [23]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5531 - accuracy: 0.7297 - 268ms/epoch - 1ms/step
Loss: 0.5530802011489868, Accuracy: 0.7297375798225403


In [24]:
# Save the model
nn.save("optimized_model.h5")

In [25]:
import xgboost as xgb

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
dtest = xgb.DMatrix(X_test_scaled, label=y_test)

# Define XGBoost parameters
param = {
    'max_depth': 5,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration (learning rate)
    'objective': 'binary:logistic',  # error evaluation for binary classification
    'eval_metric': 'error',  # evaluation metric
}
num_round = 50  # the number of training iterations

# Train the model
bst = xgb.train(param, dtrain, num_round)

# Make predictions
preds = bst.predict(dtest)
predictions = [round(value) for value in preds]

# Evaluate the model
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 72.73%


In [26]:
# Define parameter grid
param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8],
    'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8],
    'n_estimators': [50, 100, 200]
}



In [27]:
clf = xgb.XGBClassifier()

grid_search = GridSearchCV(clf, param_grid, scoring="accuracy", cv=3, verbose=1)
grid_search.fit(X_train_scaled, y_train)

# Find the best parameters and train the best model
best_xgb = grid_search.best_estimator_
best_xgb.fit(X_train_scaled, y_train)

# Predict using the best model
y_pred = best_xgb.predict(X_test_scaled)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Fitting 3 folds for each of 9375 candidates, totalling 28125 fits
Accuracy: 72.52%


In [28]:
print("Best Hyperparameters:\n", grid_search.best_params_)


Best Hyperparameters:
 {'colsample_bylevel': 0.4, 'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 0.8}


In [30]:
# the best accuracy during cross-validation
print("Best Cross-Validation Accuracy: %.2f%%" % (grid_search.best_score_ * 100))


Best Cross-Validation Accuracy: 73.14%


In [31]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on Test Data: %.2f%%" % (accuracy * 100.0))

Accuracy on Test Data: 72.52%
